In [1]:
import warnings

warnings.filterwarnings('ignore')
import numpy as _np
#from bgen_reader import open_bgen
#from os.path import join
#from pandas_plink import read_plink1_bin
#from pandas_plink import get_data_folder
# import matplotlib.pyplot as plt
#from bgen_reader import open_bgen
#from bed_reader import open_bed, sample_file
# import pandas as pd
# import os
# from pysnptools.distreader import Bgen, DistMemMap
# from pathlib import Path
from bed_reader import open_bed as _open_bed
# from pysnptools.snpreader import SnpData, SnpMemMap
# from pysnptools.util import log_in_place
# import matplotlib.pyplot as plt
# from scipy.stats import gaussian_kde
from KDEpy import FFTKDE as _FFTKDE
import pandas as _pd
# from pybgen import PyBGEN
import multiprocess as _mp

In [1]:
def MI_continuous_SNP(a,
                      b,
                      a_min,
                      a_max,
                      N=500,
                      kernel="gaussian",
                      bw="silverman"):
    """
    calculate mutual information between continuous outcome and an SNP variable of 0,1,2
    assume no missing data
    """
    # first estimate the pmf
    p0 = _np.sum(b == 0) / len(b)
    p1 = _np.sum(b == 1) / len(b)
    p2 = 1. - p0 - p1
    # estimate cond density
    _b0 = (b == 0)
    if _np.sum(
            _b0
    ) > 2:  # here proceed to kde only if there are more than 5 data points
        y_cond_p0 = _FFTKDE(kernel=kernel, bw=bw).fit(data=a[_b0])
#         y_cond_p0 = gaussian_kde(a[_b0])
    else:
        y_cond_p0 = _np.zeros_like
    _b1 = (b == 1)
    if _np.sum(_b1) > 2:
        y_cond_p1 = _FFTKDE(kernel=kernel, bw=bw).fit(data=a[_b1])
#         y_cond_p1 = gaussian_kde(a[_b1]) # this thing uses Scott's rule instead of Silverman defaulted by FFTKDE and R density
    else:
        y_cond_p1 = _np.zeros_like
    _b2 = (b == 2)
    if _np.sum(_b2) > 2:
        y_cond_p2 = _FFTKDE(kernel=kernel, bw=bw).fit(data=a[_b2])


#         y_cond_p2 = gaussian_kde(a[_b2])
    else:
        y_cond_p2 = _np.zeros_like
    joint = _np.empty((N, 3))
    a_temp = _np.linspace(a_min, a_max, num=N)
    joint[:, 0] = y_cond_p0(a_temp) * p0
    joint[:, 1] = y_cond_p1(a_temp) * p1
    joint[:, 2] = y_cond_p2(a_temp) * p2
    joint[joint < 1e-22] = 1e-22  # set a threshold to avoid numerical errors
    forward_euler_step = a_temp[1] - a_temp[0]
    #     print("total measure:", _np.sum(joint)*forward_euler_step)
    temp_log = _np.log(joint)
    #     temp_log = _np.nan_to_num(temp_log, nan = 0)
    temp1 = _np.log(_np.sum(joint, 1))
    #     temp1 = _np.nan_to_num(temp1, nan = 0)
    temp_log = temp_log - temp1.reshape(-1, 1)
    temp2 = _np.log(_np.sum(joint, 0)) + _np.log(forward_euler_step)
    #     temp2 = _np.nan_to_num(temp2, nan = 0)
    temp_log = temp_log - temp2.reshape(1, -1)
    # print(fhat_mat * temp_log)
    temp_mat = joint * temp_log
    #     temp_mat = _np.nan_to_num(temp_mat, nan=0.) # numerical fix
    mi_temp = _np.sum(temp_mat) * forward_euler_step
    return mi_temp


def MI_binary_SNP(a, b):
    """
    calculate mutual information between binary outcome and an SNP variable of 0,1,2
    assume no missing data
    """
    # first estimate the pmf of SNP
    p0 = _np.sum(b == 0) / len(b)
    p1 = _np.sum(b == 1) / len(b)
    p2 = 1. - p0 - p1
    b_marginal = _np.array([p0, p1, p2])
    # estimate pmf of the binary outcome
    a_p0 = _np.sum(a == 0) / len(a)
    a_p1 = _np.sum(a == 1) / len(a)
    a_marginal = _np.array([a_p0, a_p1]).reshape(-1, 1)
    # estimate the cond density
    joint = _np.zeros((2, 3))
    _b0 = (b == 0)
    joint[0, 0] = _np.sum(a[_b0] == 0) / len(a)
    joint[1, 0] = _np.sum(a[_b0] == 1) / len(a)
    _b1 = (b == 1)
    joint[0, 1] = _np.sum(a[_b1] == 0) / len(a)
    joint[1, 1] = _np.sum(a[_b1] == 1) / len(a)
    _b2 = (b == 2)
    joint[0, 2] = _np.sum(a[_b2] == 0) / len(a)
    joint[1, 2] = _np.sum(a[_b2] == 1) / len(a)

    _ = a_marginal * b_marginal
    _ = joint / _
    __ = joint * _np.log(_)
    __ = _np.nan_to_num(__, nan=0.0)  # for possible nuemrical issues

    mi_temp = _np.sum(__)

    return mi_temp


# make this function available
def MI_bivariate_continuous(a,
                            b,
                            a_min,
                            a_max,
                            b_min,
                            b_max,
                            a_N=300,
                            b_N=500,
                            kernel="gaussian",
                            bw="silverman",
                            norm=2):
    a_temp = _np.linspace(a_min, a_max, num=a_N)
    b_temp = _np.linspace(b_min, b_max, num=b_N)
    joint = _np.empty((a_N, b_N))
    forward_euler_step_a = a_temp[1] - a_temp[0]
    forward_euler_step_b = b_temp[1] - b_temp[0]
    data = _np.hstack(a.reshape(-1, 1), b.reshape(-1, 1))
    joint, points = FFTKDE(kernel=kernel, norm=norm).fit(data).evaluate(
        (a_temp, b_temp))
    joint[joint < 1e-22] = 1e-22
    a_forward_euler_step = a[1] - a[0]
    b_forward_euler_step = b[1] - b[0]
    log_a_marginal = _np.log(_np.sum(joint, 1)) + _np.log(b_forward_euler_step)
    log_b_marginal = _np.log(_np.sum(joint, 0)) + _np.log(a_forward_euler_step)
    log_joint = _np.log(joint)
    mi_temp = _np.sum(joint * (log_joint - log_a_marginal.reshape(
        -1, 1) - log_b_marginal)) * a_forward_euler_step * b_forward_euler_step
    return mi_temp


# make this function available
def MI_binary_continuous(a,
                         b,
                         b_min,
                         b_max,
                         N=500,
                         kernel="gaussian",
                         bw="silverman"):
    return MI_continuous_SNP(a=b,
                             b=a,
                             a_min=b_min,
                             a_max=b_max,
                             N=N,
                             kernel=kernel,
                             bw=bw)


# outcome_iid should be a  list of strings for identifiers
def continuous_filter_plink(bed_file,
                            bim_file,
                            fam_file,
                            outcome,
                            outcome_iid,
                            a_min=1.2345654321,
                            a_max=2.34565432,
                            N=500,
                            kernel="gaussian",
                            bw="silverman"):
    """
    (Single Core version) take plink files to calculate the mutual information between the continuous outcome and many SNP variables.
    """
    bed1 = _open_bed(filepath=bed_file,
                     fam_filepath=fam_file,
                     bim_filepath=bim_file)
    gene_iid = _np.array(list(bed1.iid))
    bed1_sid = _np.array(list(bed1.sid))
    outcome = outcome[_np.intersect1d(outcome_iid,
                                      gene_iid,
                                      assume_unique=True,
                                      return_indices=True)[1]]
    if a_min == 1.2345654321:
        a_min = _np.min(outcome) - _np.std(outcome)
    if a_max == 2.34565432:
        a_max = _np.max(outcome) + _np.std(outcome)
    # get genetic indices
    gene_ind = _np.intersect1d(gene_iid,
                               outcome_iid,
                               assume_unique=True,
                               return_indices=True)[1]
    MI_UKBB = _np.zeros(len(bed1_sid))
    for j in range(len(MI_UKBB)):
        _SNP = bed1.read(_np.s_[:, j], dtype=_np.int8).flatten()
        _SNP = _SNP[gene_ind]  # get gene iid also in outcome iid
        _outcome = outcome[_SNP != -127]  # remove missing SNP in outcome
        _SNP = _SNP[_SNP != -127]  # remove missing SNP
        MI_UKBB[j] = MI_continuous_SNP(a=_outcome,
                                       b=_SNP,
                                       a_min=a_min,
                                       a_max=a_max,
                                       N=N,
                                       kernel=kernel,
                                       bw=bw)
    return MI_UKBB


def binary_filter_plink(bed_file, bim_file, fam_file, outcome, outcome_iid):
    """
    (Single Core version) take plink files to calculate the mutual information between the binary outcome and many SNP variables.
    """
    bed1 = _open_bed(filepath=bed_file,
                     fam_filepath=fam_file,
                     bim_filepath=bim_file)
    gene_iid = _np.array(list(bed1.iid))
    bed1_sid = _np.array(list(bed1.sid))
    outcome = outcome[_np.intersect1d(outcome_iid,
                                      gene_iid,
                                      assume_unique=True,
                                      return_indices=True)[1]]
    # get genetic indices
    gene_ind = _np.intersect1d(gene_iid,
                               outcome_iid,
                               assume_unique=True,
                               return_indices=True)[1]
    MI_UKBB = _np.zeros(len(bed1_sid))
    print(outcome)
    for j in range(len(MI_UKBB)):
        _SNP = bed1.read(_np.s_[:, j], dtype=_np.int8).flatten()
        _SNP = _SNP[gene_ind]  # get gene iid also in outcome iid
        _outcome = outcome[_SNP != -127]  # remove missing SNP in outcome
        _SNP = _SNP[_SNP != -127]  # remove missing SNP
        MI_UKBB[j] = MI_binary_SNP(a=_outcome, b=_SNP)
    return MI_UKBB


def continuous_filter_plink_parallel(bed_file,
                                     bim_file,
                                     fam_file,
                                     outcome,
                                     outcome_iid,
                                     a_min=1.2345654321,
                                     a_max=2.34565432,
                                     N=500,
                                     core_num="NOT DECLARED",
                                     multp=1,
                                     kernel="gaussian",
                                     bw="silverman"):
    """
    (Multiprocessing version) take plink files to calculate the mutual information between the continuous outcome and many SNP variables.
    """
    if core_num == "NOT DECLARED":
        core_num = _mp.cpu_count()
    else:
        assert core_num <= _mp.cpu_count(
        ), "Declared number of cores used for multiprocessing should not exceed number of cores on this machine."
    assert core_num >= 2, "Multiprocessing should not be used on single-core machines."

    bed1 = _open_bed(filepath=bed_file,
                     fam_filepath=fam_file,
                     bim_filepath=bim_file)
    gene_iid = _np.array(list(bed1.iid))
    bed1_sid = _np.array(list(bed1.sid))
    outcome = outcome[_np.intersect1d(outcome_iid,
                                      gene_iid,
                                      assume_unique=True,
                                      return_indices=True)[1]]
    if a_min == 1.2345654321:
        a_min = _np.min(outcome) - _np.std(outcome)
    if a_max == 2.34565432:
        a_max = _np.max(outcome) + _np.std(outcome)
    # get genetic indices
    gene_ind = _np.intersect1d(gene_iid,
                               outcome_iid,
                               assume_unique=True,
                               return_indices=True)[1]

    def _continuous_filter_plink_slice(_slice):
        _MI_slice = _np.zeros(len(_slice))
        k = 0
        for j in _slice:
            _SNP = bed1.read(_np.s_[:, j], dtype=_np.int8).flatten()
            _SNP = _SNP[gene_ind]  # get gene iid also in outcome iid
            _outcome = outcome[_SNP != -127]  # remove missing SNP in outcome
            _SNP = _SNP[_SNP != -127]  # remove missing SNP
            _MI_slice[k] = MI_continuous_SNP(a=_outcome,
                                             b=_SNP,
                                             a_min=a_min,
                                             a_max=a_max,
                                             N=N,
                                             kernel=kernel,
                                             bw=bw)
            k += 1
        return _MI_slice

    # multiprocessing starts here
    ind = _np.arange(len(bed1_sid))
    with _mp.Pool(core_num) as pl:
        MI_UKBB = pl.map(_continuous_filter_plink_slice,
                         _np.array_split(ind, core_num * multp))
    MI_UKBB = _np.hstack(MI_UKBB)
    return MI_UKBB


def binary_filter_plink_parallel(bed_file,
                                 bim_file,
                                 fam_file,
                                 outcome,
                                 outcome_iid,
                                 core_num="NOT DECLARED",
                                 multp=1):
    """
    (Multiprocessing version) take plink files to calculate the mutual information between the binary outcome and many SNP variables.
    """
    if core_num == "NOT DECLARED":
        core_num = _mp.cpu_count()
    else:
        assert core_num <= _mp.cpu_count(
        ), "Declared number of cores used for multiprocessing should not exceed number of cores on this machine."
    assert core_num >= 2, "Multiprocessing should not be used on single-core machines."

    bed1 = _open_bed(filepath=bed_file,
                     fam_filepath=fam_file,
                     bim_filepath=bim_file)
    gene_iid = _np.array(list(bed1.iid))
    bed1_sid = _np.array(list(bed1.sid))
    outcome = outcome[_np.intersect1d(outcome_iid,
                                      gene_iid,
                                      assume_unique=True,
                                      return_indices=True)[1]]
    # get genetic indices
    gene_ind = _np.intersect1d(gene_iid,
                               outcome_iid,
                               assume_unique=True,
                               return_indices=True)[1]

    def _binary_filter_plink_slice(_slice):
        _MI_slice = _np.zeros(len(_slice))
        k = 0
        for j in _slice:
            _SNP = bed1.read(_np.s_[:, j], dtype=_np.int8).flatten()
            _SNP = _SNP[gene_ind]  # get gene iid also in outcome iid
            _outcome = outcome[_SNP != -127]  # remove missing SNP in outcome
            _SNP = _SNP[_SNP != -127]  # remove missing SNP
            _MI_slice[k] = MI_binary_SNP(a=_outcome, b=_SNP)
            k += 1
        return _MI_slice

    # multiprocessing starts here
    ind = _np.arange(len(bed1_sid))
    with _mp.Pool(core_num) as pl:
        MI_UKBB = pl.map(_binary_filter_plink_slice,
                         _np.array_split(ind, core_num * multp))
    MI_UKBB = _np.hstack(MI_UKBB)
    return MI_UKBB


def binary_filter_csv(csv_names=[]):
    if _np.array(csv_names).size == 0:
        print(
            "Variable names not provided -- start reading variable names from csv file now, might take some time, depending on the csv file size."
        )
        csv_names = ...
        print("Reading variable names from csv file finished.")
    else:
        csv_names = _np.array(csv_names)

    return


def continuous_filter_csv():
    pass
    return


def binary_filter_csv_parallel():
    pass
    return


def continuous_filter_csv_parallel():
    pass
    return

In [3]:
import numpy as np

a = np.random.binomial(2, .3, 200)
b = np.random.binomial(2, .3, 200)
print(MI_binary_SNP(a, b))

a = np.random.normal(200)
b = np.random.normal(200)
print(MI_binary_continuous(a, b))

0.0068276193764706585


In [4]:
import numpy as np
from bed_reader import open_bed

bed_file = r"./MI_AG/tests/sim/sim1.bed"
bim_file = r"./MI_AG/tests/sim/sim1.bim"
fam_file = r"./MI_AG/tests/sim/sim1.fam"

_bed = open_bed(filepath=bed_file,
                fam_filepath=fam_file,
                bim_filepath=bim_file)
outcome = np.random.rand(_bed.iid_count)
outcome_iid = _bed.iid

true_beta = np.array([4.2, -2.5, 2.6])
for j in np.arange(3):
    outcome += true_beta[j] * _bed.read(_np.s_[:, j], dtype=_np.int8).flatten()
    print(_bed.read(_np.s_[:, j], dtype=_np.float64).flatten())

iid_ind = np.random.permutation(np.arange(_bed.iid_count))
outcome = outcome[iid_ind]
outcome_iid = outcome_iid[iid_ind]

MI_continuous = continuous_filter_plink_parallel(bed_file=bed_file,
                                                 bim_file=bim_file,
                                                 fam_file=fam_file,
                                                 a_min=np.min(outcome) - 10,
                                                 a_max=np.max(outcome) + 10,
                                                 outcome=outcome,
                                                 outcome_iid=outcome_iid,
                                                 multp=2)

MI_continuous[:20]

[1. 0. 1. ... 2. 1. 1.]
[1. 2. 0. ... 0. 2. 1.]
[2. 1. 2. ... 2. 2. 2.]


array([0.4800993 , 0.12393376, 0.13491126, 0.00106491, 0.00214164,
       0.00115878, 0.00113526, 0.00166525, 0.00126826, 0.00142868,
       0.00163169, 0.00128086, 0.00142342, 0.00156017, 0.00159081,
       0.00189879, 0.0012721 , 0.00178034, 0.00168885, 0.00124075])

In [5]:
bed_file = r"./MI_AG/tests/sim/sim1.bed"
bim_file = r"./MI_AG/tests/sim/sim1.bim"
fam_file = r"./MI_AG/tests/sim/sim1.fam"

_bed = open_bed(filepath=bed_file,
                fam_filepath=fam_file,
                bim_filepath=bim_file)
outcome = np.random.rand(_bed.iid_count)
outcome_iid = _bed.iid

true_beta = np.array([4.2, -2.5, 2.6])
for j in np.arange(3):
    outcome += true_beta[j] * _bed.read(_np.s_[:, j], dtype=_np.int8).flatten()
    print(_bed.read(_np.s_[:, j], dtype=_np.float64).flatten())

outcome = np.random.binomial(1, np.tanh(outcome / 2) / 2 + .5)

iid_ind = np.random.permutation(np.arange(_bed.iid_count))
outcome = outcome[iid_ind]
outcome_iid = outcome_iid[iid_ind]

MI_binary = binary_filter_plink_parallel(bed_file=bed_file,
                                         bim_file=bim_file,
                                         fam_file=fam_file,
                                         outcome=outcome,
                                         outcome_iid=outcome_iid)

MI_binary[:20]

[1. 0. 1. ... 2. 1. 1.]
[1. 2. 0. ... 0. 2. 1.]
[2. 1. 2. ... 2. 2. 2.]


array([1.14262209e-01, 3.75363611e-02, 4.66570672e-02, 3.92657502e-05,
       4.73413910e-04, 1.63181674e-04, 3.97781040e-04, 4.95960688e-04,
       2.34414840e-05, 2.61841071e-04, 8.79649999e-05, 2.24676914e-04,
       5.04342096e-04, 3.54922399e-04, 1.34387451e-04, 7.05882607e-05,
       1.87493771e-04, 1.14496131e-05, 1.61946226e-04, 3.40578626e-04])